# `SENTIMENTAL ANALYSIS`

#### `IMPORTIN LIBRARIES`

In [ ]:
import pandas as pd
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout
from keras.metrics import Precision, Recall
from tensorflow.keras.optimizers import SGD
import keras.backend as K
from sklearn.preprocessing import StandardScaler

### `IMPORTING DATASET`

In [ ]:
ds=pd.read_csv("tweet_train.csv",encoding="ISO-8859-1",names=["label", "ids", "date", "flag", "user", "tweet"])
ds=ds.drop(["ids","date","flag","user"],axis=1)

In [ ]:
positive=ds[ds["label"]==0][:10000]
neagtive=ds[ds["label"]==4][:10000]

In [ ]:
positive.info()
print()
neagtive.info()

In [ ]:
positive

In [ ]:
neagtive["label"]=neagtive["label"].apply(lambda x:1)
neagtive

In [ ]:
dataset=pd.concat([positive,neagtive])
dataset=dataset.sample(frac=1)
dataset

### `PRE-PROCESSING`

##### `PRE PROCESSING VARIABLE SETTING`

In [ ]:
# dataset['tweet'] = dataset['tweet'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
dataset['tweet'] = dataset['tweet'].apply(lambda x: re.sub("@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+",' ',x))

In [ ]:
dataset

In [ ]:
dataset["tweet"]=dataset["tweet"].apply(lambda x: " ".join([w for w in x.split() if w not in stopwords.words("english")]))
dataset

In [ ]:
# Stemming
st=PorterStemmer()
dataset["tweet"]=dataset['tweet'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

In [ ]:
# Lemmatizing

dataset['tweet']=dataset['tweet'].apply(lambda x:" ".join([Word(word).lemmatize() for word in x.split()]))
dataset

### `SPLITTING DATA FOR TRAINING AND TESTING`

#### `LSTM MODEL`

In [ ]:
max_words = 1000
max_len=50

def tokenize_pad_sequences(text):
    tokenizer = Tokenizer(num_words=max_words, lower=True, split=' ')
    tokenizer.fit_on_texts(text)
    X = tokenizer.texts_to_sequences(text)
    X = pad_sequences(X, padding='post', maxlen=max_len)
    return X, tokenizer

print('Before Tokenization & Padding \n', dataset['tweet'][0])
X, tokenizer = tokenize_pad_sequences(dataset['tweet'])
print('After Tokenization & Padding \n', X[0])

In [ ]:
vocab_size = 5000
embedding_size = 32
epochs=20
learning_rate = 0.1
decay_rate = learning_rate / epochs
momentum = 0.8

sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
model= Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=max_len))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(32)))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

In [ ]:
from sklearn.model_selection import train_test_split
Y=pd.get_dummies(dataset["label"]).values
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:
def f1_score(precision, recall):
    
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
print(model.summary())

model.compile(loss='binary_crossentropy', optimizer=sgd, 
               metrics=['accuracy', Precision(), Recall()])


history = model.fit(X_train, y_train,
                      validation_data=(X_test, y_test),
                      batch_size=10, epochs=30, verbose=1)

In [ ]:
loss, accuracy, precision, recall = model.evaluate(X_test, y_test, verbose=0)
print('')
print('Accuracy  : {:.4f}'.format(accuracy))
print('Precision : {:.4f}'.format(precision))
print('Recall    : {:.4f}'.format(recall))
print('F1 Score  : {:.4f}'.format(f1_score(precision, recall)))

In [ ]:
twt = [" fucking bitch"]
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=50, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")